# SQL Options in Spark HW

Alirght let's apply what we learned in the lecture to a new dataset!

**But first!**

Let's start with Spark SQL. But first we need to create a Spark Session!

In [66]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import (
    DateType,
    StringType,
    StructField,
    StructType, 
    FloatType,
    IntegerType
)

dataset_path = 'Datasets/GooglePlayStore/'

from utils import Session

In [67]:
session = Session("section2")
spark = session.session

spark


## Read in our DataFrame for this Notebook

For this notebook we will be using the Google Play Store csv file attached to this lecture. Let's go ahead and read it in. 

### About this dataset

Contains a list of Google Play Store Apps and info about the apps like the category, rating, reviews, size, etc. 

**Source:** https://www.kaggle.com/lava18/google-play-store-apps

In [68]:
columns = [
    StructField("App", StringType(), True),
    StructField("Category", StringType(), True),
    StructField("Rating", FloatType(), True),
    StructField("Reviews", IntegerType(), True),
    StructField("Size", StringType(), True),
    StructField("Installs", StringType(), True),
    StructField("Type", StringType(), True),
    StructField("Price", FloatType(), True),
    StructField("ContentRating", StringType(), True),
    StructField("Gentes", StringType(), True),
    StructField("Last Updated", DateType(), True),
    StructField("Current Version", StringType(), True),
    StructField("Android Version", StringType(), True)

] 

schema = StructType(fields = columns)

df = spark.read.csv(
    dataset_path + "googleplaystore.csv",
    schema = schema,
    header = True
)

## First things first

Let's check out the first few lines of the dataframe to see what we are working with

In [69]:
df.limit(10).toPandas()

,App,Category,Rating,Reviews,Size,Installs,Type,Price,ContentRating,Gentes,Last Updated,Current Version,Android Version
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19M,"10,000+",Free,0.0,Everyone,Art & Design,None,1.0.0,4.0.3 and up
1,Coloring book moana,ART_AND_DESIGN,3.9,967,14M,"500,000+",Free,0.0,Everyone,Art & Design;Pretend Play,None,2.0.0,4.0.3 and up
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.7,87510,8.7M,"5,000,000+",Free,0.0,Everyone,Art & Design,None,1.2.4,4.0.3 and up
3,Sketch - Draw & Paint,ART_AND_DESIGN,4.5,215644,25M,"50,000,000+",Free,0.0,Teen,Art & Design,None,Varies with device,4.2 and up
4,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967,2.8M,"100,000+",Free,0.0,Everyone,Art & Design;Creativity,None,1.1,4.4 and up
5,Paper flowers instructions,ART_AND_DESIGN,4.4,167,5.6M,"50,000+",Free,0.0,Everyone,Art & Design,None,1.0,2.3 and up
6,Smoke Effect Photo Maker - Smoke Editor,ART_AND_DESIGN,3.8,178,19M,"50,000+",Free,0.0,Everyone,Art & Design,None,1.1,4.0.3 and up
7,Infinite Painter,ART_AND_DESIGN,4.1,36815,29M,"1,000,000+",Free,0.0,Everyone,Art & Design,None,6.1.61.1,4.2 and up
8,Garden Coloring Book,ART_AND_DESIGN,4.4,13791,33M,"1,000,000+",Free,0.0,Everyone,Art & Design,None,2.9.2,3.0 and up
9,Kids Paint Free - Drawing Fun,ART_AND_DESIGN,4.7,121,3.1M,"10,000+",Free,0.0,Everyone,Art & Design;Creativity,None,2.8,4.0.3 and up


As well as the schema to make sure all the column types were correctly infered

In [70]:
df.printSchema()

root
 |-- App: string (nullable = true)
 |-- Category: string (nullable = true)
 |-- Rating: float (nullable = true)
 |-- Reviews: integer (nullable = true)
 |-- Size: string (nullable = true)
 |-- Installs: string (nullable = true)
 |-- Type: string (nullable = true)
 |-- Price: float (nullable = true)
 |-- ContentRating: string (nullable = true)
 |-- Gentes: string (nullable = true)
 |-- Last Updated: date (nullable = true)
 |-- Current Version: string (nullable = true)
 |-- Android Version: string (nullable = true)



Looks like we need to edit some of the datatypes. We need to update Rating, Reviews and Price as integer (float for Rating) values for now, since the Size and Installs variables will need a bit more cleaning. Since we haven't been over this yet, I'm going to provide the code for you here so you can get a quick look at how it used (and how often we need it!).

**make sure to change the df name to whatever you named your df**

In [74]:

newdf = df.withColumn("Rating", df["Rating"].cast(FloatType()))             \
            .withColumn("Reviews", df["Reviews"].cast(IntegerType()))       \
            .withColumn("Price", df["Price"].cast(IntegerType()))           \
            .withColumnRenamed("Last Updated", "LastUpdated")               \
            .withColumnRenamed("Current Version", "CurrentVersion")         \
            .withColumnRenamed("Android Version", "AndroidVersion")
print(newdf.printSchema())
newdf.limit(5).toPandas()

root
 |-- App: string (nullable = true)
 |-- Category: string (nullable = true)
 |-- Rating: float (nullable = true)
 |-- Reviews: integer (nullable = true)
 |-- Size: string (nullable = true)
 |-- Installs: string (nullable = true)
 |-- Type: string (nullable = true)
 |-- Price: integer (nullable = true)
 |-- ContentRating: string (nullable = true)
 |-- Gentes: string (nullable = true)
 |-- LastUpdated: date (nullable = true)
 |-- CurrentVersion: string (nullable = true)
 |-- AndroidVersion: string (nullable = true)

None


,App,Category,Rating,Reviews,Size,Installs,Type,Price,ContentRating,Gentes,LastUpdated,CurrentVersion,AndroidVersion
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19M,"10,000+",Free,0,Everyone,Art & Design,None,1.0.0,4.0.3 and up
1,Coloring book moana,ART_AND_DESIGN,3.9,967,14M,"500,000+",Free,0,Everyone,Art & Design;Pretend Play,None,2.0.0,4.0.3 and up
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.7,87510,8.7M,"5,000,000+",Free,0,Everyone,Art & Design,None,1.2.4,4.0.3 and up
3,Sketch - Draw & Paint,ART_AND_DESIGN,4.5,215644,25M,"50,000,000+",Free,0,Teen,Art & Design,None,Varies with device,4.2 and up
4,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967,2.8M,"100,000+",Free,0,Everyone,Art & Design;Creativity,None,1.1,4.4 and up


Looks like that worked! Great! Let's dig in. 

## 1. Create Tempview

Go ahead and create a tempview of the dataframe so we can work with it in spark sql.

In [80]:
newdf.createOrReplaceTempView("newdfTempView")

## 2. Select all apps with ratings above 4.1

Use your tempview to select all apps with ratings above 4.1

In [118]:
results = spark.sql(
    "SELECT App, Category, Rating from newdfTempViw WHERE Rating > 4.1 and Rating != 'NaN'"
)

results.orderBy(("Rating")).limit(10).toPandas()

,App,Category,Rating
0,Life Made WI-Fi Touchscreen Photo Frame,1.9,19.0
1,SUMMER SONIC app,EVENTS,5.0
2,Tablet Reminder,MEDICAL,5.0
3,Prosperity,EVENTS,5.0
4,American Girls Mobile Numbers,DATING,5.0
5,Mindvalley U Tallinn 2018,EVENTS,5.0
6,Spine- The dating app,DATING,5.0
7,Eternal life,LIBRARIES_AND_DEMO,5.0
8,Online Girls Chat Group,DATING,5.0
9,Super Hearing Secret Voices Recorder PRO,MEDICAL,5.0


## 3. Now pass your results to an object 
(ie create a spark dataframe)

Select just the App and Rating column where the Category is in the Comic category and the Rating is above 4.5.

In [116]:
results = spark \
            .sql(
                "SELECT App, Category, Rating from newdfTempView WHERE Category = 'COMICS' and Rating > 4.5 AND Rating != 'NaN'"
            )\

results.orderBy(desc("Rating")).limit(10).toPandas()

,App,Category,Rating
0,Hojiboy Tojiboyev Life Hacks,COMICS,5.0
1,"Superheroes, Marvel, DC, Comics, TV, Movies News",COMICS,5.0
2,Unicorn Pokez - Color By Number,COMICS,4.8
3,WebComics,COMICS,4.8
4,GANMA! - All original stories free of charge f...,COMICS,4.7
5,Röhrich Werner Soundboard,COMICS,4.7
6,Dragon Ball Wallpaper - Ringtones,COMICS,4.7
7,Faustop Sounds,COMICS,4.7
8,"Best Wallpapers Backgrounds(100,000+ 4K HD)",COMICS,4.7
9,Manga Master - Best manga & comic reader,COMICS,4.6


## 4. Which category has the most cumulative reviews

Only select the one category with the most reivews. 

*Note: will require adding all the review together for each category*

In [124]:
# Show the top ten categories

results = spark.sql(
    "SELECT SUM(Reviews) as TotalReviews, Category\
    FROM newdfTempView\
    GROUP BY Category\
    ORDER BY TotalReviews DESC"
)

results.toPandas()

,TotalReviews,Category
0,1.585422e+09,GAME
1,8.154623e+08,COMMUNICATION
2,6.212414e+08,SOCIAL
3,4.102263e+08,FAMILY
4,2.731850e+08,TOOLS
5,2.135166e+08,PHOTOGRAPHY
6,1.150412e+08,SHOPPING
7,1.141170e+08,PRODUCTIVITY
8,1.103802e+08,VIDEO_PLAYERS
9,8.934614e+07,PERSONALIZATION


## 5. Which App has the most reviews?

Display ONLY the top result

Include only the App column and the Reviews column.

In [125]:
results.first()

Row(TotalReviews=1585422349, Category='GAME')

## 5. Select all apps that contain the word 'dating' anywhere in the title

*Note: we did not cover this in the lecture. You'll have to use your SQL knowledge :) Google it if you need to.*

In [126]:
results = spark.sql(
    "SELECT App from newdfTempView where App like '%dating%'"
)

results.limit(10).toPandas()

,App
0,"Meet, chat & date. Free dating app - Chocolate..."
1,Friend Find: free chat + flirt dating app
2,Spine- The dating app
3,Princess Closet : Otome games free dating sim
4,happn – Local dating app


## 6. Use SQL Transformer to display how many free apps there are in this list

In [127]:
from pyspark.ml.feature import SQLTransformer

In [129]:
transformer = SQLTransformer(
    statement = "SELECT COUNT(*) as FreeAppsQty FROM __THIS__ WHERE Type = 'Free'"
)

transformer.transform(newdf).show(10, False)

+-----------+
|FreeAppsQty|
+-----------+
|10037      |
+-----------+



## 7. What is the most popular Genre?

Which genre appears most often in the dataframe. Show only the top result.

In [133]:
gender_transformer = SQLTransformer(
    statement = "SELECT COUNT(*) AS GenreQty, Gentes as Genre \
                from __THIS__ group by Genre order by GenreQty desc"
)

gender_transformer.transform(newdf).toPandas()

,GenreQty,Genre
0,842,Tools
1,623,Entertainment
2,549,Education
3,463,Medical
4,460,Business
...,...,...
116,1,Trivia;Education
117,1,Music & Audio;Music & Video
118,1,Role Playing;Education
119,1,Puzzle;Education


## 8. Select all the apps in the 'Tools' genre that have more than 100 reviews

In [140]:
transformer = SQLTransformer(
    statement = "SELECT App, Gentes as Genre, Reviews \
                from __THIS__ \
                where Gentes = 'Tools' and Reviews > 100 \
                order by Reviews desc"
)

transformer.transform(newdf).toPandas()

,App,Genre,Reviews
0,Clean Master- Space Cleaner & Antivirus,Tools,42916526
1,"Security Master - Antivirus, VPN, AppLock, Boo...",Tools,24900999
2,"360 Security - Free Antivirus, Booster, Cleaner",Tools,16771865
3,DU Battery Saver - Battery Charger & Battery Life,Tools,13479633
4,Cache Cleaner-DU Speed Booster (booster & clea...,Tools,12759815
...,...,...,...
530,DNS Changer - BEST (Gprs/Edge/3G/H/H+/4G),Tools,123
531,CF cal,Tools,123
532,Evolve CP Calc. for PokemonGo,Tools,116
533,JavaScript Editor CR,Tools,114


## That's all folks! Great job!